#### Flow chart
* Strava API --> Local script (EXTRACT)
* Local script (TRANSFORM)
* Local script --> BigQuery (LOAD)
* BigQuery --> Tableau (VISUALIZE)

In [22]:
from api_calls import get_strava_data
from get_data import prepare_data, city_state_county

import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None


In [23]:
from google.cloud import bigquery
#import pandas_gbq
import pandas as pd

In [3]:
project_id = 'playground-371704'

sql = '''
select 
  NAME, 
  ADDRESS, 
  BLDG_SQFT, 
  Type_Description
FROM `grocery1.grocery`
ORDER BY 3 DESC
LIMIT 10
'''

In [4]:
client = bigquery.Client(project_id)

In [5]:
table_id = 'playground-371704.grocery1.activities'
#schema = [
 #   bigquery.SchemaField("full_name", "STRING", mode="REQUIRED"),
  #  bigquery.SchemaField("age", "STRING", mode="REQUIRED")]
table = bigquery.Table(table_id)
# table = client.create_table(table)

In [18]:
activities.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type'],
      dtype='object')

In [20]:
activities = pd.read_csv('strava_activities.csv')
activities = activities.iloc[:, 2:8]

In [21]:
client.load_table_from_dataframe(activities, table)

LoadJob<project=playground-371704, location=us-west1, id=30a8b5cf-94bc-49f8-b7bc-d217fb86bd8c>

In [57]:
# run this to update the activity data
activities_raw = get_strava_data('strava_activities.csv')

# run this to use what's saved in directory
#activities_raw = pd.read_csv('strava_activities.csv')

In [60]:
# CLEAN THE DATA
activities_clean = prepare_data(activities_raw)

In [61]:
activities_clean.start_latlng.value_counts(dropna=False)

NaN                                         15
[45.559195, -122.674891]                     1
[45.560843, -122.674843]                     1
[45.559581, -122.674898]                     1
[45.559429, -122.674865]                     1
                                            ..
[45.52720343694091, -122.60262859985232]     1
[45.52726252935827, -122.60258166119456]     1
[45.52647840231657, -122.6021517533809]      1
[45.52644135430455, -122.60234059765935]     1
[45.544204, -122.668089]                     1
Name: start_latlng, Length: 940, dtype: int64

In [42]:
test = activities_raw.start_latlng[1]

In [50]:
#        return [float(coord) for coord in elem.split('[')[1].str.split(']')[0].str.split(',')]
test[1]

return [float(coord) for coord in elem]

-122.60258870199323

In [ ]:
activities_clean[activities_clean.index > '2022-07-08']

In [ ]:
activities_clean.head()

In [ ]:
runs = activities_clean[activities_clean['sport_type'] == 'Run']
runs_2022 = runs[runs.index.year == 2022]

In [ ]:
runs_2022.loc['2022-04-06':'2022-07-07'].distance.sum()

In [ ]:
runs_2022[runs_2022.index > '2022-07-08'].distance.sum()

In [ ]:
runs_2022.columns

In [ ]:
# RUN THIS TO GET THE CITY, STATE, AND COUNTY DATA FROM THE NOMINATIM API
activities_clean = activities_clean.apply(city_state_county, axis=1)

In [ ]:
activities_clean.head()

In [ ]:
# this is a geolocation unsuccessful [43.775811, -121.515433]

location = geolocator.reverse('43.775811' + ',' + '-121.515433')


In [ ]:
location.raw['address'].get('county', 'na')

In [ ]:
activities = prepare_data()

### Strava Mobile Line Graph
Characteristics: 
* 12 weeks
* Select your activity at the top
* Header: Week of [Month day] - [Month day]
* Subheaders: 
  * Distance
  * Time in xh xxm
  * Elevation

In [ ]:
to_time_readout(runs_2022.moving_time.sum())

In [ ]:
def to_time_readout(element):
    time = element / 60 / 60
    hrs = int(time)
    mins = (time - hrs) * 60
    return f'{int(round(hrs, 0))}h {int(mins)}m'


In [ ]:
run_activities = activities[activities['sport_type'] == 'Run']
run_by_week = run_activities.resample('W-Mon', closed='left')
run_by_week_agg = run_by_week.agg({'distance': 'sum', 'moving_time': 'sum', 'total_elevation_gain': 'sum'})
run_by_week_agg['total_elevation_gain'] = run_by_week_agg['total_elevation_gain'] * 3.28084
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)
#run_by_week_agg.moving_time = pd.to_datetime(run_by_week_agg.moving_time)


In [ ]:
activities_by_week = activities.resample('W-Mon', closed='left')
activities_by_week_agg = activities_by_week.agg({'distance': 'sum', 'moving_time': 'sum', 'total_elevation_gain': 'sum'})
activities_by_week_agg['total_elevation_gain'] = activities_by_week_agg['total_elevation_gain'] * 3.28084
activities_by_week_agg.moving_time = activities_by_week_agg.moving_time.apply(to_time_readout)
#run_by_week_agg.moving_time = pd.to_datetime(run_by_week_agg.moving_time)

In [ ]:
activities_by_week_agg

In [ ]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)

In [ ]:
to_time_readout(26363)

In [ ]:
run_by_week_agg

In [ ]:
int(7.99)

In [ ]:
(7.32 - round(26363 / 60 / 60, 0)) * 60

In [ ]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(lambda x: y := x, f'{int(round(x, 0))}h {int(round((y - round(y)) * 60))}m')

In [ ]:
run_by_week_agg.moving_time[4].minutes

In [ ]:
# week of 7/11 should be 7 hrs 19m in time
#run_by_week_agg.moving_time = dt.timedelta(seconds=run_by_week_agg.moving_time)


In [ ]:
f'{int(round(time_wk, 0))}h {int(round((time_wk - round(time_wk)) * 60))}m'

In [ ]:
print(dt.timedelta(seconds=26363))

In [ ]:
# run_by_week_display = 
#run_by_week_agg.distance.round(2)
#run_by_week_agg.total_elevation_gain.round().astype(int)


In [ ]:
time_wk = 26363/60/60

In [ ]:
run_activities.iloc[[3]]

In [ ]:
# 154 ft
run_activities.total_elevation_gain[3] * 3.28084

In [ ]:
from datetime import timedelta

# create timedelta
sample_timedelta = timedelta(
    days=40,
    seconds=3,
    microseconds=2,
    milliseconds=1200,
    minutes=3,
    hours=23,
    weeks=3
)

In [ ]:
sample_timedelta.seconds

In [ ]:
#st.index = st.index.strftime('%B %Y')
st

In [ ]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)
strava_activities_clean.columns

In [ ]:
strava_activities_raw.start_latlng[109]

In [ ]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [ ]:
fig.show()

In [ ]:
type(strava_activities_clean['start_time'][2])

In [ ]:
pd_dt_obj

In [ ]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

In [ ]:
regular_dt_obj

In [ ]:
df_slice = runs_2022[runs_2022['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(12)

In [ ]:
px.line(df_slice, range_y=[0,df_slice.max()])

In [ ]:
choices = {'W-Mon': '%A, %b %d %Y', 'M': '%B %Y', 'Y': '%Y'}
choices_txt = {'By Week': 'W-Mon', 'By Month': 'M', 'By Year': 'Y'}

choice_selector = st.radio('Totals by Week, Month, or Year', choices_txt.keys())
choice = choices_txt[choice_selector]

# #strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(15)
df_slice = activities[activities['sport_type'] == 'Run'].resample(choice, closed='left').distance.sum().tail(5)
#df_slice.index = df_slice.index.strftime('%B %Y')
df_slice.index = df_slice.index.strftime(choices[choice])
st.dataframe(df_slice)

        # #strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(15)
df_slice_2 = activities[activities['sport_type'] == 'Run'].resample(choice, closed='left')[['distance', 'total_elevation_gain']].sum().tail(5)
#df_slice.index = df_slice.index.strftime('%B %Y')
df_slice_2.index = df_slice_2.index.strftime(choices[choice])
st.dataframe(df_slice_2)